In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor

from sklearn import tree
from sklearn.datasets import load_boston
import random
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report

In [91]:
boston = load_boston()
#print(boston['DESCR'])

data = boston['data']
target = boston['target'].reshape(-1,1) # 1d ro 2d array 
boston_data = np.concatenate((data,target), axis=1)
        
boston_df = pd.DataFrame(data=boston_data, columns=list(boston['feature_names'])+['MEDV'])
boston_df.head()
                             
X = boston['data']
y = boston['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)                           

In [20]:
tree_B1 = DecisionTreeRegressor(random_state=0)
tree_B1.fit(X_train, y_train)

singleTreePreds = tree_B1.predict(X_test)
np.sqrt(mean_squared_error(y_test, singleTreePreds))

4.224170737677708

## Bagging (Bootstrap aggregation)

In [21]:
baggedBoston = BaggingRegressor(base_estimator=tree_B1,n_estimators=200,oob_score=True)

In [23]:
# The folloing line of code does all of these steps:
# It fits 200 trees on 200 bootstrapped vrsions of your training data !!
baggedBoston.fit(X_train, y_train) 

# The predict function now computes 200 predistions for each data point and averages them !!
baggedPreds = baggedBoston.predict(X_test)

np.sqrt(mean_squared_error(y_test, baggedPreds))

3.808768553363686

In [24]:
#R2 
baggedBoston.oob_score_

0.8700378003192262

In [25]:
#R2 = 1-RSS/TSS = 1-MSE/ var_y
MSE = np.sqrt(mean_squared_error(y_test, baggedPreds))
var_y = np.var(y_test)

In [26]:
1-MSE/var_y

0.9528781581433466

In [30]:
baggedBoston.score(X_test, y_test)

0.8205238105598021

## Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(200, max_features=0.5, oob_score=True)
forest.fit(X_train, y_train)

forestPreds = forest.predict(X_test)
np.sqrt(mean_squared_error(y_test, forestPreds))

3.5273159618521333

### Best hyperparameter `max_features`

In [32]:
from sklearn.model_selection import GridSearchCV

In [49]:
features = ["auto", "sqrt", "log2", 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]

In [51]:
param_grid = {"max_features":features}

In [57]:
forest_grid_estimator = GridSearchCV(RandomForestRegressor(), param_grid, cv=10, n_jobs=-1, refit="neg_mean_squared_error",scoring=["r2", "neg_mean_squared_error"])


In [58]:
forest_grid_estimator.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_features': ['auto', 'sqrt', 'log2', 1, 2, 3, 4, 5,
                                          6, 7, 8, 9, 10, 11, 12, 13]},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error'])

In [70]:
forest_grid_estimator.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_features': ['auto', 'sqrt', 'log2', 1, 2, 3, 4, 5,
                                          6, 7, 8, 9, 10, 11, 12, 13]},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error'])>

In [88]:
X_test

array([[    nan,   2.6  ,  20.2  , ...,   5.757, 666.   , 100.   ],
       [    nan,   0.   ,   4.49 , ...,  18.5  , 392.3  ,   6.53 ],
       [    nan,  20.   ,   3.33 , ...,  14.9  , 387.31 ,   3.76 ],
       ...,
       [    nan,   0.   ,  19.58 , ...,  14.7  , 341.6  ,  13.28 ],
       [    nan,   0.   ,  18.1  , ...,  20.2  ,  24.65 ,  15.69 ],
       [    nan,   0.   ,  18.1  , ...,  20.2  , 393.37 ,  14.36 ]])

In [86]:
X_test_perm = X_test.copy()
X_test_perm[:,0] = np.random.shuffle(X_test_perm[:,0] )
#forestPreds_perm = forest.predict(X_test_perm)#original predictions!
#MSE_perm = np.sqrt(mean_squared_error(y_test, forestPreds_perm))#original MSE

#print("permutation importance of first feature: ", MSE_perm - MSE_orig)

In [99]:
X_test_perm = X_test.copy()

In [100]:
X_test_perm[:,0] = np.random.choice(X_test_perm[:,0] )

In [102]:
X_test_perm

array([[1.1460e-01, 0.0000e+00, 1.8100e+01, ..., 2.0200e+01, 2.6000e+00,
        1.0110e+01],
       [1.1460e-01, 0.0000e+00, 4.4900e+00, ..., 1.8500e+01, 3.9230e+02,
        6.5300e+00],
       [1.1460e-01, 2.0000e+01, 3.3300e+00, ..., 1.4900e+01, 3.8731e+02,
        3.7600e+00],
       ...,
       [1.1460e-01, 0.0000e+00, 1.9580e+01, ..., 1.4700e+01, 3.4160e+02,
        1.3280e+01],
       [1.1460e-01, 0.0000e+00, 1.8100e+01, ..., 2.0200e+01, 2.4650e+01,
        1.5690e+01],
       [1.1460e-01, 0.0000e+00, 1.8100e+01, ..., 2.0200e+01, 3.9337e+02,
        1.4360e+01]])

In [103]:
X_test

array([[5.11358e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        2.60000e+00, 1.01100e+01],
       [5.73500e-02, 0.00000e+00, 4.49000e+00, ..., 1.85000e+01,
        3.92300e+02, 6.53000e+00],
       [3.57800e-02, 2.00000e+01, 3.33000e+00, ..., 1.49000e+01,
        3.87310e+02, 3.76000e+00],
       ...,
       [1.49632e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        3.41600e+02, 1.32800e+01],
       [1.22472e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        2.46500e+01, 1.56900e+01],
       [3.56868e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.93370e+02, 1.43600e+01]])

In [ ]:
import seaborn as 